In [1]:
# Import dependencies
import pandas as pd
import requests
import json

In [2]:
# Import the api_key 
from config import api_key

In [3]:
# Create the URL 
url = "https://api.nasa.gov/neo/rest/v1/feed?"
start_date = "2023-01-01"
end_date = "2023-01-08"

In [4]:
# Assemble the query URL
query_url = f"{url}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [5]:
# Get the response
response = requests.get(query_url).json()

In [6]:
# Create a list to store the asteriod data
asteroids_list = []
asteroids = response["near_earth_objects"]
# Loop through the response and pull the data for each asteriod
for asteroid in asteroids:
    ast = response["near_earth_objects"][asteroid]
    asteroids_list.append(ast)

In [7]:
# import itertools and get seperate the list within a list
import itertools
asteroids__flist = list(itertools.chain(*asteroids_list))

In [8]:
# Put the data into a dataframe
asteroid_df = pd.DataFrame(asteroids__flist)
asteroid_df
target_cols = ["id", "name", "absolute_magnitude_h", "estimated_diameter",
               "is_potentially_hazardous_asteroid", "close_approach_data"]
asteroid_df = asteroid_df[target_cols]
asteroid_df.head()

id               name  absolute_magnitude_h  \
0  2154347  154347 (2002 XK4)                 16.08   
1  2385186  385186 (1994 AW1)                 17.67   
2  2453309  453309 (2008 VQ4)                 19.51   
3  3683468       (2014 QR295)                 18.41   
4  3703782        (2015 AE45)                 25.30   

                                  estimated_diameter  \
0  {'kilometers': {'estimated_diameter_min': 1.61...   
1  {'kilometers': {'estimated_diameter_min': 0.77...   
2  {'kilometers': {'estimated_diameter_min': 0.33...   
3  {'kilometers': {'estimated_diameter_min': 0.55...   
4  {'kilometers': {'estimated_diameter_min': 0.02...   

   is_potentially_hazardous_asteroid  \
0                              False   
1                               True   
2                              False   
3                              False   
4                              False   

                                 close_approach_data  
0  [{'close_approach_date': '2023-01-01', 'close_...  
1  [{'close_approach_date': '2023-01-01', 'close_...  
2  [{'close_approach_date': '2023-01-01', 'close_...  
3  [{'close_approach_date': '2023-01-01', 'close_...  
4  [{'close_approach_date': '2023-01-01', 'close_...

In [9]:
# Pull the data from the JSON within the "estimated_diameter" column and append the data to a list
km_min_list = []
km_max_list = []
ft_min_list = []
ft_max_list = []
for i in range(0,len(asteroid_df)):
    km_min = asteroid_df.iloc[i,3]['kilometers']["estimated_diameter_min"]
    km_min_list.append(km_min)
    km_max = asteroid_df.iloc[i,3]['kilometers']["estimated_diameter_max"]
    km_max_list.append(km_max)
    ft_min = asteroid_df.iloc[i,3]['feet']["estimated_diameter_min"]
    ft_min_list.append(ft_min)
    ft_max = asteroid_df.iloc[i,3]['feet']["estimated_diameter_max"]
    ft_max_list.append(ft_max)

In [10]:
# Pull the data from the JSON within the "close_approach_data" column and append the data to a list
kph_list = []
mph_list = []
miss_dist_list_km = []
miss_dist_miles_list = []
for i in range(0,len(asteroid_df)):
    kph = asteroid_df.iloc[i,5][0]['relative_velocity']['kilometers_per_hour']
    kph_list.append(kph)
    mph = asteroid_df.iloc[i,5][0]['relative_velocity']['miles_per_hour']
    mph_list.append(mph)
    miss_km = asteroid_df.iloc[i,5][0]['miss_distance']['kilometers']
    miss_dist_list_km.append(miss_km)
    miss_miles = asteroid_df.iloc[i,5][0]['miss_distance']['miles']
    miss_dist_miles_list.append(miss_miles)

In [11]:
# Drop any N/A values
asteroid_df.dropna(inplace=True)
# Add the columns to the dataframe
asteroid_df['km_min'] = km_min_list
asteroid_df['km_max'] = km_max_list
asteroid_df['ft_min'] = ft_min_list
asteroid_df['ft_max'] = ft_max_list
asteroid_df['velocity_kph'] = kph_list
asteroid_df['velocity_mph'] = mph_list
asteroid_df['miss_distance_km'] = miss_dist_list_km
asteroid_df['miss_distance_miles'] = miss_dist_miles_list

In [12]:
# Create the final dataframe including the new columns
target_cols = ["id", "name", "absolute_magnitude_h", "is_potentially_hazardous_asteroid",
               'km_min','km_max', 'ft_min', 'ft_max', 'velocity_kph', 'velocity_mph',
               'miss_distance_km','miss_distance_miles']
asteroid_df = asteroid_df[target_cols]
# Rename two columns
asteroid_df.rename(columns={"absolute_magnitude_h": "magnitude",
                            "is_potentially_hazardous_asteroid": "hazardous"}, inplace=True)
# Final df
asteroid_df.head()

id               name  magnitude  hazardous    km_min    km_max  \
0  2154347  154347 (2002 XK4)      16.08      False  1.616423  3.614431   
1  2385186  385186 (1994 AW1)      17.67       True  0.777240  1.737961   
2  2453309  453309 (2008 VQ4)      19.51      False  0.333085  0.744801   
3  3683468       (2014 QR295)      18.41      False  0.552783  1.236061   
4  3703782        (2015 AE45)      25.30      False  0.023150  0.051765   

        ft_min        ft_max      velocity_kph      velocity_mph  \
0  5303.224689  11858.370904  98611.9155705492  61273.6094277115   
1  2549.999104   5701.971339  46527.0874796056  28910.1227730916   
2  1092.798343   2443.571381  20959.8190961752  13023.6164822873   
3  1813.593823   4055.319071  58249.6828812893  36194.0876769878   
4    75.952142    169.834153  24703.7439103688  15349.9457647511   

     miss_distance_km  miss_distance_miles  
0  49550751.286747985   30789409.125712793  
1  33403488.139355999  20755965.0629068262  
2  39565965.365513706  24585150.8495369028  
3  39330822.646315866  24439039.9390803108  
4   8526777.284930033   5298293.7197111354

In [13]:
# Info to convert the columns
asteroid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   123 non-null    object 
 1   name                 123 non-null    object 
 2   magnitude            123 non-null    float64
 3   hazardous            123 non-null    bool   
 4   km_min               123 non-null    float64
 5   km_max               123 non-null    float64
 6   ft_min               123 non-null    float64
 7   ft_max               123 non-null    float64
 8   velocity_kph         123 non-null    object 
 9   velocity_mph         123 non-null    object 
 10  miss_distance_km     123 non-null    object 
 11  miss_distance_miles  123 non-null    object 
dtypes: bool(1), float64(5), object(6)
memory usage: 10.8+ KB


In [14]:
# Change to numeric values
asteroid_df.id = pd.to_numeric(asteroid_df.id)
asteroid_df.velocity_kph = pd.to_numeric(asteroid_df.velocity_kph)
asteroid_df.velocity_mph = pd.to_numeric(asteroid_df.velocity_mph)
asteroid_df.miss_distance_km = pd.to_numeric(asteroid_df.miss_distance_km)
asteroid_df.miss_distance_miles = pd.to_numeric(asteroid_df.miss_distance_miles)
# New Info
asteroid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   123 non-null    int64  
 1   name                 123 non-null    object 
 2   magnitude            123 non-null    float64
 3   hazardous            123 non-null    bool   
 4   km_min               123 non-null    float64
 5   km_max               123 non-null    float64
 6   ft_min               123 non-null    float64
 7   ft_max               123 non-null    float64
 8   velocity_kph         123 non-null    float64
 9   velocity_mph         123 non-null    float64
 10  miss_distance_km     123 non-null    float64
 11  miss_distance_miles  123 non-null    float64
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 10.8+ KB


In [21]:
# Import the dependencies
import numpy as np
from sqlalchemy import create_engine
# make sure to create config file in vscode
from config1 import password

In [29]:
# connect to local database
protocol = 'postgresql'
username = 'postgres'
host = 'localhost'
port = 5432               
database_name = 'Project_3_Asteriods'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [27]:
# check for table
engine.table_names()

C:\Users\brads\AppData\Local\Temp\ipykernel_15392\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['asteriod_df']

In [28]:
# Use pandas to load data into the database
asteroid_df.to_sql(name='asteriod_df', con=engine, if_exists='append', index=False)

123